### [T&uuml;rme von Hanoi](https://de.wikipedia.org/wiki/T%C3%BCrme_von_Hanoi)
Ein Scheibenstapel soll verschoben werden. Dabei sind folgende Regeln einzuhalten:
- Es darf jeweils nur eine Scheibe verschoben werden.
- Auf eine Scheibe darf nur eine kleinere Scheibe gelegt werden.
- Es dürfen max. 3 Stapel vorhanden sein.

Wir programmieren eine einfache, aber ausbauf&auml;hige Variante dieses Spiels.  
- Verschoben wird ein Stabel mit 4 Scheiben,
  repräsentiert durch die Zahlen 0, 1, 2 und 3. 
  Je grösser die Zahl,
  desto grösser die Scheibe.
- Eine absteigend sortierte Liste `[3, 2, 1, 0]` entspricht einem Stapel.
  Die letzte Zahl entspricht der obersten und kleinsten Scheibe.

### Der Kern des Spiels
Der Spielzustand wird in der Variable `stacks` gespeichert.
Diese enthält eine Liste mit 3 Listen, den Stapeln.
Die Funktion `new_game()` startet ein neues Spiel, indem die Liste `stacks` modifiziert wird, so dass sie die 3 Stapel  
`[3, 2, 1, 0]`, `[]` und `[]`  
enthält.

Die Funktion `move_disk(src, dst)` verschiebt eine Scheibe von Stapel `src` (source) auf den Stapel `dst` (destination), falls möglich (**besser**: falls regelkonform).  
Die Funktion  `new_game()` und `move_disk(src, dst)` geben zudem den aktuellen Spielzustand aus.

In [ ]:
ndisks = 4
stacks = []


def show_stacks(stacks):
    print(stacks)


def new_game():
    stack_1 = list(range(ndisks))[::-1]
    stacks[:] = [stack_1, [], []]
    show_stacks(stacks)


def move_disk(src, dst):
    if not stacks[src]:
        return
    disk = stacks[src].pop()
    stacks[dst].append(disk)
    show_stacks(stacks)

In [ ]:
new_game()

In [ ]:
move_disk(0, 2)

### Bildliche Darstellung der Stapelkonfiguration
Statt einer textlichen Darstellung der Stapelkonfiguration
sollen soll nun die Funktion `show_stacks(stacks)` eine bildliche Darstellung liefern.
Zu desem Zweck schreiben wir eine Funktion `make_drawStacks_cmd(stacks)`,
die einen String liefert, welcher unseren Zeichungsautomaten ein entsprechendes Bild zeichnen lässt.

Wir zerlegen des Problem in kleinere Teilprobleme. Zuerst
erstellen wir  die Zeichungsanweisung für eine einzelne Scheibe,
dann für einen einzelnen Stapel und schliesslich für alle Stapel.




In [ ]:
disks_config = {
    'colors': ('brown', 'teal', 'blue', 'purple'),
    'widths': (30, 50, 70, 90),
    'height': 10,
    'xpositions': (50, 150, 250),
}

In [ ]:
def make_drawDisk_cmd(disk, col, row):
    '''disk: 0, 1, 2 oder 3
       col: x-Position (0, 1 oder 2)
       row: y-Position (0, 1, 2 oder 3)
    '''
    cmds = []
    width = disks_config['widths'][disk]
    height = disks_config['height']
    color = disks_config['colors'][disk]
    x = disks_config['xpositions'][col] - width/2
    y = 100 - row*height

    cmds.append(f'f{color};')
    cmds.append('u')
    cmds.append(f'g{x},{y};')
    cmds.append(f'R{width},{height};')

    return ''.join(cmds)

In [ ]:
make_drawDisk_cmd(2, 0, 1)

In [ ]:
def make_drawStack_cmd(stack, col):
    cmd = ''.join(make_drawDisk_cmd(disk, col, i) for i, disk in enumerate(stack))
    return cmd

In [ ]:
make_drawStack_cmd((3, 0), 2)

In [ ]:
def make_drawStacks_cmd(stacks):
    cmd = ''.join(make_drawStack_cmd(stack, i) for i, stack in enumerate(stacks))
    return cmd

In [ ]:
make_drawStacks_cmd([[2, 1], [3, 0], []])

In [ ]:
import zeichnungsautomat as ZA


automat, drawingBoard = ZA.get_automat_and_canvas(width=300, height=200)

In [ ]:
def show_stacks(stacks):
    canvas.clear()
    cmd = make_drawStacks_cmd(stacks)
    automat.read(cmd)

In [ ]:
drawingBoard

In [ ]:
new_game()

In [ ]:
move_disk(0, 2)